# **Especialização em Ciência de Dados - INF/UFRGS e SERPRO**
### Disciplina CD003 - Aprendizado Supervisionado
#### *Profa. Mariana Recamonde-Mendoza (mrmendoza@inf.ufrgs.br)*
<br> 

---
***Observação:*** *Este notebook é disponibilizado aos alunos como complemento às aulas síncronas e aos slides preparados pela professora. Desta forma, os principais conceitos são apresentados no material teórico fornecido. O objetivo deste notebook é reforçar os conceitos e demonstrar questões práticas no uso de diferentes algoritmos e estratégias de Aprendizado de Máquina.*


---



<br>

## **Aula 03** - **Tópico: Algoritmo Naive Bayes**

<br>

O algoritmo **Naive Bayes** é um algoritmo probabilístico que assume que os valores dos atributos de uma instância são independentes entre si dada a classe. Assim, ao determinar a probabilidade a posteriori $P(y_i|\textbf{x})$ para uma determinada classe $i$ e uma instância $\textbf{x}$ através do Teorema de Bayes, o algoritmo calcula $P(\textbf{x}|y_i)$ como o produto das probabilidades condicionais de cada atributo individualmente ($x^i, x^2, ..., x^d$) considerando a classe $i$.

A simplicidade na implementação do algoritmo, o fato de representar o seu conhecimento através de probabilidades que são compreensíveis por humanos, a ausência de (muitos) hiperparâmetros que demandam ajustes, e a sua robustez a ruídos, tornam o **Naive Bayes** um algoritmo bastante interesse para domínios que apresentam incerteza e como uma primeira escolha de algoritmo para abordar novos problemas de classificação. Além disso, é um algoritmo que tende a se adaptar melhor em problemas com alta dimensionalidade (número de atributos >> número de instâncias).
<br> 

**Objetivo deste notebook**: Compreender o uso do algoritmo Naive Bayes em problemas de classificação e analisar soluções para lidar com conjuntos de dados que apresentam tipos de atributos mistos (numéricos e categóricos). 

<br>

---





##**Analisando o risco de doenças cardíacas**

De acordo com o Centers for Disease Control and Prevention (CDC) dos Estados Unidos, a doença cardíaca é uma das principais causas de morte para pessoas da maioria das raças nos EUA (afro-americanos, índios americanos e nativos do Alasca e pessoas brancas). Cerca de metade de todos os americanos (47%) têm pelo menos 1 dos 3 principais fatores de risco para doenças cardíacas: pressão alta, colesterol alto e tabagismo. Outros indicadores-chave incluem estado diabético, obesidade (IMC alto), não praticar atividade física suficiente ou beber muito álcool. Detectar e prevenir os fatores que têm maior impacto nas doenças cardíacas é muito importante na área da saúde. 

Os dados a serem analisados neste notebook foram disponibilizados no Kaggle e derivam do *Behavioral Risk Factor Surveillance System (BRFSS)*, que realiza pesquisas telefônicas anuais para coletar dados sobre o estado de saúde dos residentes dos EUA. Como o CDC descreve: *"Estabelecido em 1984 com 15 estados, o BRFSS agora coleta dados em todos os 50 estados, bem como no Distrito de Columbia e três territórios dos EUA. O BRFSS completa mais de 400.000 entrevistas com adultos a cada ano, tornando-se o maior sistema de pesquisa no mundo."*. O conjunto de dados mais recente (em 15 de fevereiro de 2022) inclui dados de 2020. A grande maioria das colunas são perguntas feitas aos entrevistados sobre seu estado de saúde, como "Você tem sérias dificuldades para andar ou subir escadas?" ou "Você fumou pelo menos 100 cigarros em toda a sua vida? [Nota: 5 maços = 100 cigarros]". A grande base de dados gerada pela pesquisa realizada pelo CDC foi filtrada, de forma a manter fatores diferentes (perguntas) que influenciam direta ou indiretamente as doenças cardíacas.

Nesta atividade, iremos utilizar o algoritmo de Naive Bayes para prever a chance de uma pessoa ter doença cardíaca a partir de um conjunto de perguntas que se relacionam a fatores de risco da doença. Os dados a serem utilizados foram filtrados pela professora, a fim de manter um subconjunto de **60 mil** instâncias.




---



###Carregando e inspecionando os dados

Primeiramente, vamos carregar algumas bibliotecas importantes do Python e os dados a serem utilizados neste estudo. Os dados são disponibilizados através de um link, que também pode ser diretamente acessado pelos alunos.

In [ ]:
## Carregando as bibliotecas necessárias
# A primeira linha é incluída para gerar os gráficos logo abaixo dos comandos de plot
%matplotlib inline              
import pandas as pd             # para análise de dados 
import matplotlib.pyplot as plt # para visualização de informações
import seaborn as sns           # para visualização de informações
import numpy as np              # para operações com arrays multidimensionais
from sklearn.naive_bayes import GaussianNB # para treinar NB com dados contínuos
from sklearn.naive_bayes import CategoricalNB # para treinar NB com dados categóricos
from sklearn.model_selection import train_test_split # para divisão de dados
from sklearn.metrics import confusion_matrix, recall_score, precision_score,accuracy_score,ConfusionMatrixDisplay ## para avaliação dos modelos


sns.set()


In [ ]:
df = pd.read_csv("https://drive.google.com/uc?export=view&id=1WOW3Q3IL6bTbrNkyP1YhdMbsU31I_R2g")
df  

In [ ]:
## Características gerais do dataset
print("O conjunto de dados possui {} linhas e {} colunas".format(df.shape[0], df.shape[1]))

A coluna *'HeartDisease'* contém a classificação de cada amostra referente a ocorrência de doença cardíaca. Vamos avaliar como as instâncias estão distribuídas entre as classes presentes no dataset.

In [ ]:
## Distribuição do atributo alvo
plt.hist(df['HeartDisease'])
plt.title("Distribuição do atributo alvo")
plt.show()

Também é importante averiguar os tipos de dados de cada coluna (atributo), bem como se existem valores faltantes. Os valores faltantes normalmente estão codificados como NaN e serão identificados pelo Python com o comando `isnull()`. Caso estejam codificados de outra forma, é necessário substituir por NaN.

In [ ]:
df.info()

## Para analisar valores faltantes, quando codificados como NaN, podemos usar o
## comando abaixo
df.isnull().sum()

Podemos perceber que existem atributos do tipo categórico (object) e do tipo numérico (int64). Vamos analisar cada um dos tipos, bem como dividir em variáveis distintas para facilitar a manipulação dos dados.

In [ ]:
## Encontrar as variáveis categóricas
categorical = [var for var in df.columns if df[var].dtype=='O']

## Heart disease é categórica, mas se trata do atributo alvo - removemos da lista
categorical = categorical [1:len(categorical)]

print('Existem {} atributos categóricos no conjunto de dados\n'.format(len(categorical)))

print('Os atributos categóricos são :', categorical)

Para os atributos categóricos, podemos investigar quantos valores diferentes cada um deles pode assumir, bem como a distribuição das classes entre as categorias de cada atributo.

In [ ]:
##Verificar a cardinalidade dos valores categóricos 
for var in categorical:
    print(var, ' contains ', len(df[var].unique()), ' labels')

In [ ]:
## Gerar um gráfico para cada variável categórica com a distribuição de 
## frequência entre as classes
def count_plot(df,columns,label):
    plt.figure(figsize=(16, 10))
    for indx, var  in enumerate(columns):
        plt.subplot(4, 4, indx+1)
        g = sns.countplot(x=var, data=df, hue=label)
    plt.tight_layout()

count_plot(df, categorical,'HeartDisease')

Vamos olhar com mais atenção os atributos AgeCategory, Race e Diabetic, pois possuem mais categorias que não puderam ser bem visualizadas no gráfico acima. A célula abaixo executa para o primeiro atributo. Modifique e analise para os demais atributos mencionados.

In [ ]:
plt.figure(figsize=(16, 10))

#sns.countplot(x='AgeCategory', data=df, hue='HeartDisease')
#sns.countplot(x='Race', data=df, hue='HeartDisease')
sns.countplot(x='Diabetic', data=df, hue='HeartDisease')



**Responda >>>** Observe a frequência de HeartDisease para cada variável categórica. Alguma variável parece ter um desequilíbrio na frequência das classes para os diferentes valores que pode assumir?

> ***Sua resposta aqui:*** No gráfico por Race, há muito mais ocorrências para brancos (White) do que não brancos, e também não parece haver ocorrências nos dados para o caso asiático (Asian) com doença cardíaca (HeartDisease = Yes). No gráfico por Diabetic, há poucas ocorrências paras os casos de pré-diabetes (borderline diabetes) e durante a gravidez (during pregnancy)

Vamos repetir a análise agora para os atributos numéricos. O código abaixo identifica as variáveis numéricas e plota as suas distribuições de valores.

In [ ]:
## Encontrar as variáveis numéricas

numerical = [var for var in df.columns if df[var].dtype!='O']

print('Existem {} atributos numéricos no conjunto de dados\n'.format(len(numerical)))

print('Os atributos numéricos são :', numerical)

In [ ]:
## Gerar um gráfico para cada variável numérica com a distribuição de 
## frequência entre as classes
def dist_plot(df,columns,label):
    plt.figure(figsize=(16, 10))
    for indx, var  in enumerate(columns):
        plt.subplot(2, 2, indx+1)
        g = sns.histplot(x=var, data=df, hue=label,binwidth=3)
    plt.tight_layout()

dist_plot(df, numerical,'HeartDisease')


---


### Criando conjuntos de treino e teste para avaliação de modelos


Antes de iniciar o treinamento do modelo, vamos aplicar o método holdout e separar uma porção dos dados para teste.

O algoritmo Naive Bayes tem várias implementações no scikit-learn. Vamos adotar duas delas ao longo da atividade: CategoricalNB e GaussianNB, onde a primeira aceita como entrada somente dados categóricos e a segunda somente dados numéricos. Um dos pré-requisitos para uso da CategoricalNB() é que os dados categóricos estejam codificados em valores numéricos (ao invés de [Baixo, Médio, Alto], representar os valores por [0, 1, 2], por exemplo). Assim, vamos aplicar esta codificação e então dividir os dados em conjuntos de treino (80%) e teste (20%).



In [ ]:
from sklearn.preprocessing import OrdinalEncoder # para converter variáveis categóricas (strings para inteiros)

## Separa o dataset em duas variáveis: os atributos/entradas (X) e a classe/saída (y)
## Originalmente a classe se encontra na primeira coluna ('HeartDisease')
X = df.drop(['HeartDisease'], axis=1)
y = df['HeartDisease'].values

# Codifica variáveis categóricas usando inteiros, pré-requisito para CategoricalNB()
encoder = OrdinalEncoder(dtype=np.int64)
encoder.fit(X[categorical])
X[categorical] = encoder.transform(X[categorical])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,stratify=y,random_state=42) ## fixe um número de random_state para facilitar a reprodutibilidade

---


### Treinamento de modelos individuais para dados categóricos e numéricos

Vamos fazer agora dois treinamentos distintos, primeiro um modelo com base nos atributos categóricos e em seguida um modelo com base nos atributos numéricos.

Iniciamos separando os conjuntos de dados para os dois tipos de atributos.

In [ ]:
## Identificamos os índices das colunas para cada tipo de atributo
catFeat_idx = np.argwhere(X.columns.isin(categorical)).ravel()
numFeat_idx = np.argwhere(X.columns.isin(numerical)).ravel()

## A partir dos índices, separamos os atributos usados no treino e no teste
## A classe (y) não precisa ser separada, pois é a mesma para ambos
X_train_cat = X_train.iloc[:,catFeat_idx]
X_train_num = X_train.iloc[:,numFeat_idx]

X_test_cat = X_test.iloc[: ,catFeat_idx]
X_test_num = X_test.iloc[: ,numFeat_idx]

Abaixo definimos uma funçao auxiliar para visualizar a fronteira de decisão dos classificadores, semelhante à que foi usada na atividade anterior.

In [ ]:
## Fonte: https://github.com/tirthajyoti/Machine-Learning-with-Python/blob/master/Utilities/ML-Python-utils.py
def plot_decision_boundaries(X, y, model_class, **model_params):
    """
    Function to plot the decision boundaries of a classification model.
    This uses just the first two columns of the data for fitting 
    the model as we need to find the predicted value for every point in 
    scatter plot.
    Arguments:
            X: Feature data as a NumPy-type array.
            y: Label data as a NumPy-type array.
            model_class: A Scikit-learn ML estimator class 
            e.g. GaussianNB (imported from sklearn.naive_bayes) or
            LogisticRegression (imported from sklearn.linear_model)
            **model_params: Model parameters to be passed on to the ML estimator
    
    Typical code example:
            plt.figure()
            plt.title("KNN decision boundary with neighbros: 5",fontsize=16)
            plot_decision_boundaries(X_train,y_train,KNeighborsClassifier,n_neighbors=5)
            plt.show()
    """
    try:
        X = np.array(X)
        y = np.array(y).flatten()
    except:
        print("Coercing input data to NumPy arrays failed")

    # Reduces to the first two columns of data - for a 2D plot!
    reduced_data = X[:, :2]
    # Instantiate the model object
    model = model_class(**model_params)
    # Fits the model with the reduced data
    model.fit(reduced_data, y)

    # Step size of the mesh. Decrease to increase the quality of the VQ.
    h = .02     # point in the mesh [x_min, m_max]x[y_min, y_max].    

    # Plot the decision boundary. For that, we will assign a color to each
    x_min, x_max = reduced_data[:, 0].min() - 0.5, reduced_data[:, 0].max() + 0.5
    y_min, y_max = reduced_data[:, 1].min() - 0.5, reduced_data[:, 1].max() + 0.5
    # Meshgrid creation
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    # Obtain labels for each point in mesh using the model.
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])    

    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))

    # Predictions to obtain the classification results
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    # Plotting
    plt.contourf(xx, yy, Z, alpha=0.2,cmap='viridis')
    g=plt.scatter(X[:, 0], X[:, 1], c=y, alpha=0.6,s=50, edgecolor='k', cmap='viridis' )
    plt.xlabel("Feature-1",fontsize=15)
    plt.ylabel("Feature-2",fontsize=15)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(handles=g.legend_elements()[0],labels=('No','Yes'))
    return plt

#### Modelo Naive Bayes com dados numéricos

Iniciamos criando um modelo Naive Bayes para os dados numéricos, usando o método `GaussianNB()`. Este modelo usa apenas 4 atributos, para lembrarmos:

In [ ]:
print(numerical)

In [ ]:
## Treinando o modelo com GaussianNB
nbG_clf = GaussianNB()
nbG_clf.fit(X_train_num,y_train)
print(nbG_clf.class_prior_)

## Prevendo a classe de saída para as instâncias de teste
## Por padrão, a classe retornada é aquela que maximiza a probabilidade a posteriori
y_predG = nbG_clf.predict(X_test_num)

## As probabilidades por classe podem ser observadas com o seguinte comando
y_probaG = nbG_clf.predict_proba(X_test_num)
print(y_probaG[:10])

Avaliando o desempenho do modelo Naive Bayes Gaussiano.

In [ ]:
cm = confusion_matrix(y_test, y_predG,labels=nbG_clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=nbG_clf.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print('Acurácia: {}'.format(round(accuracy_score(y_test, y_predG),3)))
print('Recall: {}'.format(round(recall_score(y_test, y_predG,pos_label='Yes'),3)))
print('Precisão: {}'.format(round(precision_score(y_test, y_predG,pos_label='Yes'),3)))

#####Observando a fronteira de decisão do modelo NB Gaussiano


In [ ]:
NUM_INSTANCES = 500
y_train_int =  np.array([0 if y=='No' else 1 for y in y_train]) 

plot_decision_boundaries(X_train_num[:NUM_INSTANCES],y_train_int[:NUM_INSTANCES],GaussianNB)

**>>>> Responda:** Como a fronteira de decisão muda para diferentes tamanhos de subconjuntos de instâncias? Altere o valor de NUM_INSTANCES, tentando valores menores e maiores do que o valor inicial.

***Sua resposta aqui:*** Testei valores 40, 30, 20 e 10 abaixo de 50, e valores 60, 70, 90, 100, 150, 200, acima de 50.

Foi possível perceber a linha ficar cada vez menos curva e se assemalhar com um reta para valores muito altos.

#### Modelo Naive Bayes com dados categóricos

Agora vamos treinar um modelo Naive Bayes para os dados categóricos, usando o método `CategoricalNB()`.

In [ ]:
## Treinando o modelo com CategoricalNB
nbC_clf = CategoricalNB(alpha=1.0e-10)
nbC_clf.fit(X_train_cat,y_train)

print(nbC_clf.class_log_prior_)

## Prevendo a classe de saída para as instâncias de teste
## Por padrão, a classe retornada é aquela que maximiza a probabilidade a posteriori
y_predC = nbC_clf.predict(X_test_cat)

## As probabilidades por classe podem ser observadas com o seguinte comando
y_probaC = nbC_clf.predict_proba(X_test_cat)
print(y_probaC[:10])



Avaliando o desempenho do modelo Naive Bayes Categórico

In [ ]:
cm = confusion_matrix(y_test, y_predC,labels=nbC_clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=nbC_clf.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print('Acurácia: {}'.format(round(accuracy_score(y_test, y_predC),3)))
print('Recall: {}'.format(round(recall_score(y_test, y_predC,pos_label='Yes'),3)))
print('Precisão: {}'.format(round(precision_score(y_test, y_predC,pos_label='Yes'),3)))

**>>>> Exercício:** Como se comparam os desempenhos dos dois modelos em relação às métricas análisadas e ao número de FP, FN, TP, TN?

***Sua resposta aqui:*** 

Naive Bayes Gaussiano, apresentou um número muito elevado de falso positivo, um número muito acima dos verdadeiros postivos.

Em relação aos valores de acurácia, recall e precisão o algoritimo Naive Bayes Categórico apresentou resultados melhores.

#####Observando a fronteira de decisão do modelo NB Categórico

In [ ]:
NUM_INSTANCES = 500
y_train_int =  np.array([0 if y=='No' else 1 for y in y_train]) 

plot_decision_boundaries(X_train_cat.iloc[:NUM_INSTANCES,:],y_train_int[:NUM_INSTANCES],CategoricalNB)



---


### Naive Bayes com dados Categóricos e Numéricos - Abordagem 1

Uma abordagem possível para utilizar dados de atributos mistos (categóricos e numéricos) é realizar o treinamento separadamente e agregar as probabilidades de cada modelo para tomar a decisão final através da multiplicação.

Neste caso, é importante notar que as probabilidades retornadas pela função `predict_proba()` já incluem a multiplicação pela probabilidade a priori. Ao agregarmos esta probabilidade, é preciso corrigir a duplicidade no uso da probabilidade a priori das classes. Assim, seguimos os seguintes passos


1.   Treinamos cada modelo separadamente (exercício anterior)
2.   Obtemos as probabilidades preditas para cada instância de teste com cada modelo, obtendo y_probaC e y_probaG.
3.   Multiplicamos as probabilidades obtidas para cada classificador, para cada instância/classe
4.    Corrigimos a multiplicação, dividindo pela probabilidade a priori das classes
5.    Normalizamos os resultados obtidos (para cada instância a soma deve ser 1)



Para esta abordagem, vamos utilizar os modelos individuais previamente treinados e suas respectivas probabilidades calculadas para os exemplos de teste.

In [ ]:
from sklearn.preprocessing import normalize

## O que vamos calcular:
## ProbGaussian * (ProbCategorical/PriorProb)
## Normalizando os resultados para que a probabilidade para cada instância some 1 (norm=l1_)
normed_matrix = normalize(y_probaC*(y_probaG/nbG_clf.class_prior_), axis=1, norm='l1')
#print(np.sum(normed_matrix,axis=1))

## Analisamos a coluna (0 ou 1) com valor máximo
max_index_col = np.argmax(normed_matrix, axis=1).tolist()

## Mapeamos os valores inteiros (0 ou 1) para o nome das classes
y_predM2 = np.array(['No' if y==0 else 'Yes' for y in max_index_col]) 

cm = confusion_matrix(y_test, y_predM2,labels=nbC_clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=nbC_clf.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print('Acurácia: {}'.format(round(accuracy_score(y_test, y_predM2),3)))
print('Recall: {}'.format(round(recall_score(y_test, y_predM2,pos_label='Yes'),3)))
print('Precisão: {}'.format(round(precision_score(y_test, y_predM2,pos_label='Yes'),3)))



---


### Naive Bayes com dados Categóricos e Numéricos - Abordagem 2

Outra abordagem possível para treinar um modelo Naive Bayes usando dados mistos (categóricos e numéricos) é discretizar os atributos numéricos usando um processo de "*binning*". Vamos usar o conceito de percentil para discretizar os dados numéricos. Para o atributo BMI (IMC, índice de massa corporal), usaremos os pontos de cortes estabelecidos na interpretação deste índice.

In [ ]:
## Separa o dataset em duas variáveis: os atributos/entradas (X) e a classe/saída (y)
## Originalmente a classe se encontra na primeira coluna ('HeartDisease')
X = df.drop(['HeartDisease'], axis=1)
y = df['HeartDisease'].values

## Discretizar variáveis contínuas, divindindo a distribuição em 5 bins (20% dos dados)
## Armazenar os limites para cada threshold (e.g., PHcutoffs), para ser aplicado posteriormente
X['PhysicalHealth'],PHcutoffs =pd.cut(x=X['PhysicalHealth'],bins=5, labels=['bottom20', 'lower20', 'middle20', 'upper20', 'top20'],retbins=True)
X['MentalHealth'],MHcutoffs=pd.cut(x=X['MentalHealth'], bins=5, labels=['bottom20', 'lower20', 'middle20', 'upper20', 'top20'],retbins=True)
X['SleepTime'],STcutoffs=pd.cut(x=X['SleepTime'], bins=5,labels=['bottom20', 'lower20', 'middle20', 'upper20', 'top20'],retbins=True)


## Discretizar BMI usando pontos de corte padrão (9999 garante abranger todos os valores possíveis)
X['BMI']=pd.cut(x=X['BMI'], bins=[0, 18.5, 24.9, 29.9,9999], labels = ['slim', 'normal', 'overweight', 'obese'])

## Codifica variáveis categóricas, pré-requisito para CategoricalNB
encoder = OrdinalEncoder(dtype=np.int64)
encoder.fit(X)
X = encoder.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,stratify=y,random_state=42)

## Treinando o modelo de NB Misto
nbM1_clf = CategoricalNB(alpha=1e-10)
nbM1_clf.fit(X_train,y_train)
#print(nbM1_clf.class_log_prior_)

## Prevendo a classe de saída para as instâncias de teste
## Por padrão, a classe retornada é aquela que maximiza a probabilidade a posteriori
y_predM1 = nbM1_clf.predict(X_test)

## As probabilidades por classe podem ser observadas com o seguinte comando
y_probaM1 = nbM1_clf.predict_proba(X_test)
#print(y_probaM1[:10])


## Avaliando o desempenho do modelo usando a matriz de confusão, e três métricas 
## de desempenho: acurácia, recall (sensibilidade) e precisão.

cm = confusion_matrix(y_test, y_predM1,labels=nbM1_clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=nbM1_clf.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print('Acurácia: {}'.format(round(accuracy_score(y_test, y_predM1),3)))
print('Recall: {}'.format(round(recall_score(y_test, y_predM1,pos_label='Yes'),3)))
print('Precisão: {}'.format(round(precision_score(y_test, y_predM1,pos_label='Yes'),3)))



---

### Aplicando o modelo NB para novos dados

**>>>> Exercício:** Assuma que agora um novo arquivo foi liberado com a resposta de novas pessoas que responderam à pesquisa do CDC. Faça o tratamento apropriado dos dados e aplique um dos modelos NB treinados (por exemplo, o modelo NB Misto com a Abordagem 2). Após realizar a predição, avalie os erros detectados em relação aos valores de atributos das instâncias de entrada, averiguando a possibilidade de erros sistemáticos no modelo treinado.

Dica: caso deseje usar a Abordagem 2, você pode utilizar os cutoffs calculados anteriormente para discretizar os atributos. por exemplo:
`X2['PhysicalHealth']=pd.cut(x=X2['PhysicalHealth'],bins=PHcutoffs, labels=['bottom20', 'lower20', 'middle20', 'upper20', 'top20'])`



















In [ ]:
df_test = pd.read_csv("https://drive.google.com/uc?export=view&id=1oWoYNOF8rBzZcWivff6ae4eAAwafNvGQ")

X2 = df_test.drop(['HeartDisease'], axis=1)
y2 = df_test['HeartDisease'].values

#df_test


#### 
# Adicione aqui o código para pré-processar os dados de acordo com o modelo escolhido
# e para realizar a predição das novas instâncias.

## Identificamos os índices das colunas para cada tipo de atributo
catFeat_idx = np.argwhere(X2.columns.isin(categorical)).ravel()
numFeat_idx = np.argwhere(X2.columns.isin(numerical)).ravel()

# Codifica variáveis categóricas usando inteiros, pré-requisito para CategoricalNB()
encoder = OrdinalEncoder(dtype=np.int64)
encoder.fit(X2[categorical])
X2[categorical] = encoder.transform(X2[categorical])

## A partir dos índices, separamos os atributos por cat ou num
X2_test_cat = X2.iloc[: ,catFeat_idx]
X2_test_num = X2.iloc[: ,numFeat_idx]


# modelo com GaussianNB
print('Resultado para variáveis numéricas')
y2_predG = nbG_clf.predict(X2_test_num)
y2_probaG = nbG_clf.predict_proba(X2_test_num) # probabilidades

cm = confusion_matrix(y2, y2_predG,labels=nbG_clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=nbG_clf.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print('Acurácia: {}'.format(round(accuracy_score(y2, y2_predG),3)))
print('Recall: {}'.format(round(recall_score(y2, y2_predG,pos_label='Yes'),3)))
print('Precisão: {}'.format(round(precision_score(y2, y2_predG,pos_label='Yes'),3)))

# modelo com CategoricalNB
print('Resultado para variáveis categóricas')
y2_predC = nbC_clf.predict(X2_test_cat)
y2_probaC = nbC_clf.predict_proba(X2_test_cat) # probabilidades

cm = confusion_matrix(y2, y2_predC,labels=nbC_clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=nbC_clf.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print('Acurácia: {}'.format(round(accuracy_score(y2, y2_predC),3)))
print('Recall: {}'.format(round(recall_score(y2, y2_predC,pos_label='Yes'),3)))
print('Precisão: {}'.format(round(precision_score(y2, y2_predC,pos_label='Yes'),3)))

print('Resultado total')
## O que vamos calcular:
## ProbGaussian * (ProbCategorical/PriorProb)
## Normalizando os resultados para que a probabilidade para cada instância some 1 (norm=l1_)
normed_matrix = normalize(y2_probaC*(y2_probaG/nbG_clf.class_prior_), axis=1, norm='l1')
#print(np.sum(normed_matrix,axis=1))

## Analisamos a coluna (0 ou 1) com valor máximo
max_index_col = np.argmax(normed_matrix, axis=1).tolist()

## Mapeamos os valores inteiros (0 ou 1) para o nome das classes
y2_predM2 = np.array(['No' if y==0 else 'Yes' for y in max_index_col]) 

cm = confusion_matrix(y2, y2_predM2,labels=nbC_clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=nbC_clf.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print('Acurácia: {}'.format(round(accuracy_score(y2, y2_predM2),3)))
print('Recall: {}'.format(round(recall_score(y2, y2_predM2,pos_label='Yes'),3)))
print('Precisão: {}'.format(round(precision_score(y2, y2_predM2,pos_label='Yes'),3)))


# Ao final, você pode observar os acertos/erros da seguinte forma:
output = pd.DataFrame({'Pred':y2_predM2, 'Real':y2,'Correct':y2_predM2==y2})
print(output)




####




---


#### Sugestões de experimentos extras:

*   Para os experimentos com os dados disponibilizados em arquivo independente, verifique se remover atributos sensíveis como *Race* impacta nos resultados. Também verifique se estratificar atributos categóricos de outra forma (menos categorias, por exemplo) pode aprimorar os resultados
*   Teste outras alternativas para agregar a saída dos classificadores Naive Bayes Gaussiano e Categórico. Por exemplo, tente uma votação majoritária entre ambos, ou uma votação majoritária ponderada pelo número de atributos avaliado por cada modelo.
*   Explore diferentes valores para o hiperparâmetro $alpha$ do CategoricalNB, e verifique se impacta nos resultados obtidos